In [1]:
import locale
import os

In [ ]:
!pip install git+https://github.com/salesforce/BLIP.git
!pip install torchvision

In [ ]:
!pip install voluptuous

In [4]:
!pip install diffusers==0.21.4 huggingface-hub==0.16.4 transformers==4.31.0 accelerate==0.21.0 xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.30.1
    Uninstalling huggingface-hub-0.30.1:
      Successfully uninstalled huggingface-hub-0.30.1
  A

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts

In [26]:
import os
os.makedirs("/content/training_data/narrative-diffusion", exist_ok=True)
os.makedirs("/content/training_data/narrative-diffusion/100_avatar", exist_ok=True)

In [27]:
from google.colab import files
from PIL import Image
from io import BytesIO

# Step 1: Create directory
save_dir = "/content/training_data/narrative-diffusion/100_avatar"
os.makedirs(save_dir, exist_ok=True)

# Step 2: Upload images
uploaded_files = files.upload()

# Step 3: Save and rename files
for i, (filename, filedata) in enumerate(uploaded_files.items(), start=1):
    # Get image format
    extension = os.path.splitext(filename)[-1].lower()
    if extension not in ['.jpg', '.jpeg', '.png', '.webp']:
        extension = '.png'  # default if unknown

    # Load image
    img = Image.open(BytesIO(filedata))

    # Create new filename
    new_filename = f"image{i}{extension}"
    save_path = os.path.join(save_dir, new_filename)

    # Save image
    img.save(save_path)

print(f"\nAll images saved and renamed in: {save_dir}")


Saving image1.jpg to image1.jpg
Saving image2.jpg to image2.jpg
Saving image3.jpg to image3.jpg
Saving image4.jpg to image4.jpg
Saving image5.jpg to image5.jpg
Saving purple_girl_Back-view.JPG to purple_girl_Back-view.JPG
Saving purple_girl_Close-up Full-body.JPG to purple_girl_Close-up Full-body.JPG
Saving purple_girl_Front-view.JPG to purple_girl_Front-view.JPG
Saving purple_girl_Full-body.JPG to purple_girl_Full-body.JPG
Saving purple_girl_raw.jpg to purple_girl_raw.jpg
Saving purple_girl_Side-view.JPG to purple_girl_Side-view.JPG

All images saved and renamed in: /content/training_data/narrative-diffusion/100_avatar


In [28]:
from PIL import Image
from pathlib import Path
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Set the path to your avatar images
image_dir = Path("/content/training_data/narrative-diffusion/100_avatar")

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

# Caption each image and save .txt file
for image_path in image_dir.glob("*.jpg"):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    txt_path = image_path.with_suffix(".txt")
    txt_path.write_text(caption)
    print(f"{image_path.name} → {caption}")


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


image6.jpg → a woman in a blue top and black pants
image5.jpg → a woman in a blue top and black pants
image3.jpg → a woman in a blue top and black pants
image9.jpg → a cartoon character with long hair and a blue top
image8.jpg → a woman in a blue top and black pants
image11.jpg → a woman in a blue top and black pants
image2.jpg → a cartoon character with long hair and a blue shirt
image4.jpg → a cartoon character with long hair and a blue top
image7.jpg → a cartoon character with long hair and a blue shirt
image1.jpg → a woman in a blue top and black pants
image10.jpg → a set of female poses in various poses


In [29]:
import json

metadata = [{
    "instance_prompt": "photo of avatar",
    "class_prompt": "photo of a character",
    "image": f"/content/training_data/narrative-diffusion/{img}"
} for img in os.listdir("/content/training_data/narrative-diffusion")]

with open("meta.json", "w") as f:
    json.dump(metadata, f, indent=4)


In [31]:
import subprocess

command = [
    "accelerate", "launch", "/content/sd-scripts/train_network.py",
    "--pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5",
    "--train_data_dir=/content/training_data/narrative-diffusion",
    "--network_module=networks.lora",
    "--output_dir=/content/output_lora",
    "--logging_dir=/content/logs",
    "--resolution=512,512",
    "--train_batch_size=1",
    "--max_train_steps=100",
    "--save_model_as=safetensors",
    "--learning_rate=1e-4",
    "--mixed_precision=fp16",
    "--caption_extension=.txt",
    "--network_dim=8",
    "--output_name=narrative-diffusion-lora",
    "--cache_latents",
    "--prior_loss_weight=1.0"
]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding="utf-8", errors="replace")

for line in process.stdout:
    print(line, end='')


2025-04-08 06:54:19.055208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744095259.217352   15281 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744095259.266908   15281 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-04-08 06:54:30.742403: E external/local_xla/xla/s

In [20]:
import subprocess

subprocess.run(["pip", "install", "-q", "diffusers", "transformers", "accelerate", "safetensors", "xformers"], check=True)


CompletedProcess(args=['pip', 'install', '-q', 'diffusers', 'transformers', 'accelerate', 'safetensors', 'xformers'], returncode=0)

In [22]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Paths
base_model = "runwayml/stable-diffusion-v1-5"
lora_path = "/content/output_lora/narrative-diffusion-lora.safetensors"

# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    safety_checker=None,
).to("cuda")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Load your LoRA weights
pipe.load_lora_weights(lora_path)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [24]:
from diffusers import StableDiffusionPipeline
import torch
from safetensors.torch import load_file
from PIL import Image
import os

# Path to your LoRA
lora_path = "/content/output_lora/narrative-diffusion-lora.safetensors"

# Load base Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
).to("cuda")

# Load LoRA weights
state_dict = load_file(lora_path)
pipe.unet.load_state_dict(state_dict, strict=False)

# Prompt
prompt = "a purple anime girl standing in a fantasy forest, full body, masterpiece, 4k"

# Generate
image = pipe(prompt).images[0]
image.save("output.png")
image.show()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


  0%|          | 0/50 [00:00<?, ?it/s]